# Interactive OpenAI Function Calling Notebook

Welcome to the Interactive Function Calling Notebook, tailored to guide you step-by-step through Open AI function. 

Creator: [Josh Bickett](https://twitter.com/josh_bickett)

Colab: [Interactive Notebook](https://colab.research.google.com/github/joshbickett/function-calling-notebook/blob/main/interactive-function-calling-notebook.ipynb)

Github: [Interactive Notebook](https://github.com/joshbickett/function-calling-notebook)

## What is a Function?

Functions let ChatGPT do things in the real world.



In [ ]:
!pip install openai

In [3]:

import requests
import openai
import json
import random

In [4]:
openai.api_key = 'YOUR_API_KEY'

In [5]:

def launch_water_balloon(balloon_size):
    hit_locations = ['the ground', 'a tree', 'the moon']
    random_location = random.choice(hit_locations)
    launch_results = f'Launching the {balloon_size} water balloon, it hit {random_location}!'
    return launch_results



In [6]:
functions = [
    {
        "name": "launch_water_balloon",
        "description": "This function launches a hypothetical water balloon",
        "parameters": {
            "type": "object",
            "properties": {
                "balloon_size": {
                    "type": "string",
                    "enum": ["Large", "Medium", "Small"],
                },
            },
            "required": ["balloon_size"],
        },
    }
]

available_functions = {
    "launch_water_balloon": launch_water_balloon,
}

In [ ]:
INSPECT = True

In [13]:

def converse(messages):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
    )
    response_message = response["choices"][0]["message"]
    
    
    messages.append(response_message) 
    if response_message.get("function_call"):
        function_name = response_message["function_call"]["name"]
        function_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        if INSPECT:
          print('[inspector] Oh, we got a function call! 🔨')
          print('[inspector] ===> function_name', function_name)
          print('[inspector] ===> function_args', function_args)
        function_response = function_to_call(
            balloon_size=function_args.get("balloon_size"),
        )
        if INSPECT:
          print('[inspector] ===> function_response', function_response)


        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )

        print('Assistant: ', function_response)
        
    else: 
        content = response_message.get("content")
        print('Assistant: ', content)

    

      

In [ ]:
messages = []
message_count = 0 
while message_count < 10: 
  user_input = input("User: ")
  user_message = {"role": "user", "content": user_input}
  messages.append(user_message)
  converse(messages)
  message_count += 1